# Fitting of TCV q-Profiles

This notebook performs polynomial and piecewise linear fits of the safety factor (q) profiles for TCV plasmas, and compares the resulting fits for q-profile, magnetic shear, and shift. The workflow includes visualization and extraction of fit coefficients for further use.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Context_PT:
    a_shift = 0.25  # Parameter in Shafranov shift
    Z_axis = 0.1414361745  # Magnetic axis height [m]
    R_axis = 0.8727315068  # Magnetic axis major radius [m]
    B_axis = 1.4  # Magnetic field at the magnetic axis [T]
    R_LCFSmid = 1.0968432365089495  # Major radius of the LCFS at the outboard midplane [m]
    x_inner = 0.04  # Radial extent inside LCFS
    x_outer = 0.08  # Radial extent outside LCFS
    Rmid_min = R_LCFSmid - x_inner  # Minimum midplane major radius of simulation box [m]
    Rmid_max = R_LCFSmid + x_outer  # Maximum midplane major radius of simulation box [m]
    R0 = 0.5 * (Rmid_min + Rmid_max)  # Major radius of the simulation box [m]
    a_mid = R_LCFSmid - R_axis  # Minor radius at outboard midplane [m]
    # Redefine a_mid with Shafranov shift, to ensure LCFS radial location
    a_mid = R_axis / a_shift - np.sqrt(R_axis * (R_axis - 2 * a_shift * R_LCFSmid + 2 * a_shift * R_axis)) / a_shift
    r0 = R0 - R_axis  # Minor radius of the simulation box [m]
    Lx = Rmid_max - Rmid_min
    x_min = 0.0
    x_max = Lx
    Ly = 0.2
    Lz = 2.0 * np.pi - 1e-10
    Nx = 48
    
    def r_x(self, x):
        return x + self.a_mid - self.x_inner
    
ctx = Context_PT()

def qprofile_TCV_PT(r, ctx):
    y = r + ctx.R_axis
    a = 497.3420166252413
    b = -1408.736172826569
    c = 1331.4134861681464
    d = -419.00692601227627
    return (a*y**3 + b*y**2 + c*y + d)

qprofile_TCV = qprofile_TCV_PT

def shear(r, qprof, ctx):
    q = qprof(r, ctx)  # Assuming q is callable
    dqdr = np.gradient(q, r)
    shear = (r/q) * dqdr
    return shear

def shift(x, qprofile, ctx):
    x0 = 0.5 * (ctx.x_min + ctx.x_max)
    r0 = ctx.r_x(x0)
    q0 = qprofile(r0, ctx)  # Assuming qprofile is callable
    r = ctx.r_x(x)
    return -ctx.r0 / q0 * qprofile(r,ctx) * ctx.Lz


x = np.linspace(ctx.x_min, ctx.x_max, ctx.Nx) # radial coordinate
r = ctx.r_x(x) # minor radius
R = ctx.R_axis + r # major radius

# linear fit of the TCV q-profile
qprofile = qprofile_TCV(r, ctx)
qfit_lin = np.polyfit(R, qprofile, 1)
def qfit_lin_func(r, ctx):
    y = r + ctx.R_axis
    return 1.5 * y 
    return qfit_lin[0] * y + qfit_lin[1]
# print the coefficients of the linear fit
print("Linear fit coefficients: ", qfit_lin)

# quadratic fit of the TCV q-profile
qfit_quad = np.polyfit(R, qprofile, 2)
def qfit_quad_func(r, ctx):
    y = r + ctx.R_axis
    return qfit_quad[0] * y**2 + qfit_quad[1] * y + qfit_quad[2]
# print the coefficients of the quadratic fit
print("Quadratic fit coefficients: ", qfit_quad)

# cubic fit of the TCV q-profile
qfit_cub = np.polyfit(R, qprofile, 3)
def qfit_cub_func(r, ctx):
    y = r + ctx.R_axis
    return qfit_cub[0] * y**3 + qfit_cub[1] * y**2 + qfit_cub[2] * y + qfit_cub[3]

# piecewise linear fit of the TCV q-profile
Npieces = 8
qfit_piecewise = np.zeros((Npieces, 2))
for i in range(Npieces):
    x1 = i * (ctx.x_max - ctx.x_min) / Npieces
    x2 = (i + 1) * (ctx.x_max - ctx.x_min) / Npieces
    r1 = ctx.r_x(x1)
    r2 = ctx.r_x(x2)
    R1 = r1 + ctx.R_axis
    R2 = r2 + ctx.R_axis
    q1 = qprofile_TCV(r1, ctx)
    q2 = qprofile_TCV(r2, ctx)
    qfit_piecewise[i, 0] = (q2 - q1) / (R2 - R1)  # slope
    qfit_piecewise[i, 1] = q1 - qfit_piecewise[i, 0] * R1  # intercept

    
def qfit_piecewise_func(r, ctx):
    if isinstance(r, float): 
        r = np.array([r])
    y = r + ctx.R_axis
    fit = np.zeros_like(y)
    for i in range(len(y)):
        for j in range(Npieces):
            x1 = j * (ctx.x_max - ctx.x_min) / Npieces
            x2 = (j + 1) * (ctx.x_max - ctx.x_min) / Npieces
            R1 = ctx.r_x(x1) + ctx.R_axis
            R2 = ctx.r_x(x2) + ctx.R_axis
            if y[i] >= R1 and y[i] <= R2:
                fit[i] = qfit_piecewise[j, 0] * y[i] + qfit_piecewise[j, 1]
    return fit
    
# print the coefficients of the piecewise linear fit and their interval in a c code format with 16 digits
print("Piecewise linear fit coefficients in C code format: ")
for i in range(Npieces):
    x1 = i * (ctx.x_max - ctx.x_min) / Npieces
    x2 = (i + 1) * (ctx.x_max - ctx.x_min) / Npieces
    r1 = ctx.r_x(x1)
    r2 = ctx.r_x(x2)
    R1 = r1 + ctx.R_axis
    R2 = r2 + ctx.R_axis
    print("if (R >= %.14g && R <= %.14g) q = %.14g * R + %.14g;" % (R1, R2, qfit_piecewise[i, 0], qfit_piecewise[i, 1]))


# compare the fits and the original TCV data for q profile, shear and shift
fig, ax = plt.subplots(1, 3, figsize=(12, 5))
# Plot q-profile
ax[0].plot(R, qprofile, '-k', label='Original TCV')
ax[0].plot(R, qfit_lin_func(r, ctx), '-', label='Linear Fit')
# ax[0].plot(R, qfit_quad_func(r, ctx), '-', label='Quadratic Fit')
# ax[0].plot(R, qfit_cub_func(r, ctx), '-', label='Cubic Fit')
ax[0].plot(R, qfit_piecewise_func(r, ctx), '--c', label='Piecewise Linear Fit')
ax[0].set_xlabel('Major Radius [m]')
ax[0].set_ylabel('q-profile')
ax[0].set_title('q-profile')
# Plot shear
shear_original = shear(r, qprofile_TCV, ctx)
shear_linear = shear(r, qfit_lin_func, ctx)
shear_quadratic = shear(r, qfit_quad_func, ctx)
ax[1].plot(R, shear_original, '-k')
ax[1].plot(R, shear_linear, '-')
# ax[1].plot(R, shear_quadratic, '-')
# ax[1].plot(R, shear(r, qfit_cub_func, ctx), '-')
ax[1].plot(R, shear(r, qfit_piecewise_func, ctx), '--c')
ax[1].set_xlabel('Major Radius [m]')
ax[1].set_ylabel(r'$r/q(r) \times \partial q(r)/\partial r$')
ax[1].set_title('Shear')
# Plot shift
shift_original = shift(x, qprofile_TCV, ctx)/ctx.Ly
shift_linear = shift(x, qfit_lin_func, ctx)/ctx.Ly
shift_quadratic = shift(x, qfit_quad_func, ctx)/ctx.Ly
ax[2].plot(R, shift_original, '-k')
ax[2].plot(R, shift_linear, '-')
# ax[2].plot(R, shift_quadratic, '-')
# ax[2].plot(R, shift(x, qfit_cub_func, ctx)/ctx.Ly, '-')
ax[2].plot(R, shift(x, qfit_piecewise_func, ctx)/ctx.Ly, '--c')
ax[2].set_xlabel('Major Radius [m]')
ax[2].set_ylabel(r'$S_y/L_y$')
ax[2].set_title('Shift')
# Add legend only to the first subplot
ax[0].legend()#loc='upper center', bbox_to_anchor=(1.5, 1.2), ncol=3)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Context_NT:
    a_shift = 0.5  # Parameter in Shafranov shift
    Z_axis = 0.1414361745  # Magnetic axis height [m]
    R_axis = 0.8867856264  # Magnetic axis major radius [m]
    B_axis = 1.4  # Magnetic field at the magnetic axis [T]
    R_LCFSmid = 1.0870056099999  # Major radius of the LCFS at the outboard midplane [m]
    x_inner = 0.04  # Radial extent inside LCFS
    x_outer = 0.08  # Radial extent outside LCFS
    Rmid_min = R_LCFSmid - x_inner  # Minimum midplane major radius of simulation box [m]
    Rmid_max = R_LCFSmid + x_outer  # Maximum midplane major radius of simulation box [m]
    R0 = 0.5 * (Rmid_min + Rmid_max)  # Major radius of the simulation box [m]
    a_mid = R_LCFSmid - R_axis  # Minor radius at outboard midplane [m]
    # Redefine a_mid with Shafranov shift, to ensure LCFS radial location
    a_mid = R_axis / a_shift - np.sqrt(R_axis * (R_axis - 2 * a_shift * R_LCFSmid + 2 * a_shift * R_axis)) / a_shift
    r0 = R0 - R_axis  # Minor radius of the simulation box [m]
    Lx = Rmid_max - Rmid_min
    x_min = 0.0
    x_max = Lx
    Ly = 0.2
    Lz = 2.0 * np.pi - 1e-10
    Nx = 48
    
    def r_x(self, x):
        return x + self.a_mid - self.x_inner
    
ctx = Context_NT()

def qprofile_TCV_NT(r, ctx):
    y = r + ctx.R_axis
    a = 484.0615913225881
    b = -1378.25993228584
    c = 1309.3099150729233
    d = -414.13270311478726
    return (a*y**3 + b*y**2 + c*y + d)

qprofile_TCV = qprofile_TCV_NT

def shear(r, qprof, ctx):
    q = qprof(r, ctx)  # Assuming q is callable
    dqdr = np.gradient(q, r)
    shear = (r/q) * dqdr
    return shear

def shift(x, qprofile, ctx):
    x0 = 0.5 * (ctx.x_min + ctx.x_max)
    r0 = ctx.r_x(x0)
    q0 = qprofile(r0, ctx)  # Assuming qprofile is callable
    r = ctx.r_x(x)
    return -ctx.r0 / q0 * qprofile(r,ctx) * ctx.Lz


x = np.linspace(ctx.x_min, ctx.x_max, ctx.Nx) # radial coordinate
r = ctx.r_x(x) # minor radius
R = ctx.R_axis + r # major radius

# linear fit of the TCV q-profile
qprofile = qprofile_TCV(r, ctx)
qfit_lin = np.polyfit(R, qprofile, 1)
def qfit_lin_func(r, ctx):
    y = r + ctx.R_axis
    return 1.5 * y 
    return qfit_lin[0] * y + qfit_lin[1]
# print the coefficients of the linear fit
print("Linear fit coefficients: ", qfit_lin)

# quadratic fit of the TCV q-profile
qfit_quad = np.polyfit(R, qprofile, 2)
def qfit_quad_func(r, ctx):
    y = r + ctx.R_axis
    return qfit_quad[0] * y**2 + qfit_quad[1] * y + qfit_quad[2]
# print the coefficients of the quadratic fit
print("Quadratic fit coefficients: ", qfit_quad)

# cubic fit of the TCV q-profile
qfit_cub = np.polyfit(R, qprofile, 3)
def qfit_cub_func(r, ctx):
    y = r + ctx.R_axis
    return qfit_cub[0] * y**3 + qfit_cub[1] * y**2 + qfit_cub[2] * y + qfit_cub[3]

# piecewise linear fit of the TCV q-profile
Npieces = 8
qfit_piecewise = np.zeros((Npieces, 2))
for i in range(Npieces):
    x1 = i * (ctx.x_max - ctx.x_min) / Npieces
    x2 = (i + 1) * (ctx.x_max - ctx.x_min) / Npieces
    r1 = ctx.r_x(x1)
    r2 = ctx.r_x(x2)
    R1 = r1 + ctx.R_axis
    R2 = r2 + ctx.R_axis
    q1 = qprofile_TCV(r1, ctx)
    q2 = qprofile_TCV(r2, ctx)
    qfit_piecewise[i, 0] = (q2 - q1) / (R2 - R1)  # slope
    qfit_piecewise[i, 1] = q1 - qfit_piecewise[i, 0] * R1  # intercept

    
def qfit_piecewise_func(r, ctx):
    if isinstance(r, float): 
        r = np.array([r])
    y = r + ctx.R_axis
    fit = np.zeros_like(y)
    for i in range(len(y)):
        for j in range(Npieces):
            x1 = j * (ctx.x_max - ctx.x_min) / Npieces
            x2 = (j + 1) * (ctx.x_max - ctx.x_min) / Npieces
            R1 = ctx.r_x(x1) + ctx.R_axis
            R2 = ctx.r_x(x2) + ctx.R_axis
            if y[i] >= R1 and y[i] <= R2:
                fit[i] = qfit_piecewise[j, 0] * y[i] + qfit_piecewise[j, 1]
    return fit
    
# print the coefficients of the piecewise linear fit and their interval in a c code format with 16 digits
print("Piecewise linear fit coefficients in C code format: ")
for i in range(Npieces):
    x1 = i * (ctx.x_max - ctx.x_min) / Npieces
    x2 = (i + 1) * (ctx.x_max - ctx.x_min) / Npieces
    r1 = ctx.r_x(x1)
    r2 = ctx.r_x(x2)
    R1 = r1 + ctx.R_axis
    R2 = r2 + ctx.R_axis
    print("if (R >= %.14g && R <= %.14g) q = %.14g * R + %.14g;" % (R1, R2, qfit_piecewise[i, 0], qfit_piecewise[i, 1]))


# compare the fits and the original TCV data for q profile, shear and shift
fig, ax = plt.subplots(1, 3, figsize=(12, 5))
# Plot q-profile
ax[0].plot(R, qprofile, '-k', label='Original TCV')
ax[0].plot(R, qfit_lin_func(r, ctx), '-', label='Linear Fit')
# ax[0].plot(R, qfit_quad_func(r, ctx), '-', label='Quadratic Fit')
# ax[0].plot(R, qfit_cub_func(r, ctx), '-', label='Cubic Fit')
ax[0].plot(R, qfit_piecewise_func(r, ctx), '--c', label='Piecewise Linear Fit')
ax[0].set_xlabel('Major Radius [m]')
ax[0].set_ylabel('q-profile')
ax[0].set_title('q-profile')
# Plot shear
shear_original = shear(r, qprofile_TCV, ctx)
shear_linear = shear(r, qfit_lin_func, ctx)
shear_quadratic = shear(r, qfit_quad_func, ctx)
ax[1].plot(R, shear_original, '-k')
ax[1].plot(R, shear_linear, '-')
# ax[1].plot(R, shear_quadratic, '-')
# ax[1].plot(R, shear(r, qfit_cub_func, ctx), '-')
ax[1].plot(R, shear(r, qfit_piecewise_func, ctx), '--c')
ax[1].set_xlabel('Major Radius [m]')
ax[1].set_ylabel(r'$r/q(r) \times \partial q(r)/\partial r$')
ax[1].set_title('Shear')
# Plot shift
shift_original = shift(x, qprofile_TCV, ctx)/ctx.Ly
shift_linear = shift(x, qfit_lin_func, ctx)/ctx.Ly
shift_quadratic = shift(x, qfit_quad_func, ctx)/ctx.Ly
ax[2].plot(R, shift_original, '-k')
ax[2].plot(R, shift_linear, '-')
# ax[2].plot(R, shift_quadratic, '-')
# ax[2].plot(R, shift(x, qfit_cub_func, ctx)/ctx.Ly, '-')
ax[2].plot(R, shift(x, qfit_piecewise_func, ctx)/ctx.Ly, '--c')
ax[2].set_xlabel('Major Radius [m]')
ax[2].set_ylabel(r'$S_y/L_y$')
ax[2].set_title('Shift')
# Add legend only to the first subplot
ax[0].legend()#loc='upper center', bbox_to_anchor=(1.5, 1.2), ncol=3)
plt.tight_layout()
plt.show()
